#### 0. Check for GPU

In [ ]:
!nvidia-smi

In [ ]:
a = []
while(1):
    a.append('1')

#### 1. Install the necessary dependencies

In [38]:
!pip install --quiet transformers
!pip install --quiet pytorch-lightning
!pip install --quiet tokenizers
!pip install --quiet sentencepiece

You should consider upgrading via the 'C:\Users\aviparna.biswas\Anaconda3\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\aviparna.biswas\Anaconda3\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\aviparna.biswas\Anaconda3\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\aviparna.biswas\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [39]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

In [40]:
import pandas as pd
import torch
import numpy as np
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
from termcolor import colored
import textwrap

In [41]:
import os
from torch import nn
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torch.utils.data import random_split
from torchvision import transforms

In [42]:
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

In [43]:
pl.seed_everything(42)

42

#### 2. Import Dataset

In [ ]:
!gdown --id 1C-MJgB8kRb97NK97hyn4LFhuTZE4aFdw

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1C-MJgB8kRb97NK97hyn4LFhuTZE4aFdw
To: /content/ARC.zip
100% 2.02k/2.02k [00:00<00:00, 2.87MB/s]


In [ ]:
!unzip -q ARC.zip

In [67]:
with Path("ARC/arc-train-factoid-1c.json").open(encoding = "utf-8") as json_file:
  data = json.load(json_file)

In [68]:
data.keys()

dict_keys(['data', 'version'])

In [69]:
data["version"]

'ARC_Q&A'

In [70]:
data["data"][0].keys()

dict_keys(['paragraphs', 'title'])

In [71]:
data["data"][0]["title"]

'ARC_Q&A'

In [72]:
data["data"][0]["paragraphs"]

[{'qas': [{'id': 'ARC_QUE_063',
    'question': 'Was the Aspire opened on the correct mortgage account?',
    'answers': [{'text': 'Yes', 'answer_start': 0}]}],
  'context': 'Yes 1 No 0'},
 {'qas': [{'id': 'ARC_QUE_064',
    'question': 'Was the Aspire opened on an M&amp;T Serviced Loan, M&amp;T returned check, and/or Insurance check?',
    'answers': [{'text': 'Yes', 'answer_start': 0}]}],
  'context': 'Yes 1 No 0'},
 {'qas': [{'id': 'ARC_QUE_065',
    'question': 'Was the Aspire opened to the correct team?',
    'answers': [{'text': 'Yes', 'answer_start': 0}]}],
  'context': 'Yes 1 No 0'},
 {'qas': [{'id': 'ARC_QUE_066',
    'question': 'Was the check copy and / or documentation attached to the Aspire?',
    'answers': [{'text': 'Yes', 'answer_start': 0}]}],
  'context': 'Yes 1 No 0'},
 {'qas': [{'id': 'ARC_QUE_067',
    'question': 'If instructions were given outside the Aspire Case, was documentation attached evidencing the instructions? (Internal Chat, Email, ETC.)',
    'answers'

In [73]:
len(data["data"][0]["paragraphs"])

13

In [74]:
questions = data["data"][0]["paragraphs"]

In [75]:
questions[0]

{'qas': [{'id': 'ARC_QUE_063',
   'question': 'Was the Aspire opened on the correct mortgage account?',
   'answers': [{'text': 'Yes', 'answer_start': 0}]}],
 'context': 'Yes 1 No 0'}

#### 3. Defining our model

In [76]:
def extract_questions_and_answers(factoid_path: Path):
    with factoid_path.open() as json_file:
        data = json.load(json_file)

    questions = data["data"][0]["paragraphs"]

    data_rows = []

    for question in questions:
        context = question["context"]
        for question_and_answers in question["qas"]:
            question = question_and_answers["question"]
            answers = question_and_answers["answers"]

            for answer in answers:
                answer_text = answer["text"]
                answer_start = answer["answer_start"]
                answer_end = answer_start + len(answer_text)

                data_rows.append({
                    "question":question,
                    "context":context,
                    "answer_text":answer_text,
                    "answer_start":answer_start,
                    "answer_end":answer_end
                })

    return pd.DataFrame(data_rows)

In [77]:
extract_questions_and_answers(Path("ARC/arc-train-factoid-1a.json")).head()

,question,context,answer_text,answer_start,answer_end
0,Did the LO brand PHH Mortgage?,PASS/ 1 FAIL/0 NA/0,PASS,0,4
1,Did the LO identify themselves by First and La...,PASS/ 1 FAIL/0 NA/0,PASS,0,4
2,Did the LO give the customer their NMLS ID #?,PASS/ 1 FAIL/0 NA/0,PASS,0,4
3,Did the LO state the following: 'This call is ...,PASS/ 1 FAIL/0 NA/0,PASS,0,4
4,Did the LO obtain last 4 of SSN and property a...,PASS/ 1 FAIL/0 NA/0,PASS,0,4


In [78]:
factoid_paths = sorted(list(Path("ARC/").glob("arc-train-*")))
factoid_paths

[WindowsPath('ARC/arc-train-factoid-1a.json'),
 WindowsPath('ARC/arc-train-factoid-1b.json'),
 WindowsPath('ARC/arc-train-factoid-1c.json')]

In [85]:
dfs = []

for factoid_path in factoid_paths:
    dfs.append(extract_questions_and_answers(factoid_path))

df = pd.concat(dfs)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 6504: character maps to <undefined>

In [20]:
df.head()

NameError: name 'df' is not defined

In [ ]:
df.shape

In [ ]:
len(df.question.unique())

In [ ]:
len(df.context.unique())

In [ ]:
sample_question = df.iloc[21]
sample_question

In [ ]:
def color_answer(question):
  answer_start, answer_end = question["answer_start"], question["answer_end"]
  context = question["context"]

  return colored(context[:answer_start], "red") + \
    colored(context[answer_start:answer_end + 1], "green") + \
    colored(context[answer_end + 1:], "red")

In [ ]:
print(sample_question["question"])
print()
print("Answer:")

for wrap in textwrap.wrap(color_answer(sample_question), width=120):
  print(wrap)

#### 5. Tokenization

In [ ]:
MODEL_NAME = 't5-base'

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

In [ ]:
encoding = tokenizer(
    sample_question["question"],
    sample_question["context"],
    max_length=396,
    padding="max_length",
    truncation="only_second",
    return_attention_mask = True,
    add_special_tokens = True,
    return_tensors = "pt"
)

In [ ]:
encoding.keys()

In [ ]:
tokenizer.special_tokens_map

In [ ]:
tokenizer.eos_token, tokenizer.eos_token_id

In [ ]:
tokenizer.decode(encoding["input_ids"].squeeze())

In [ ]:
answer_encoding = tokenizer(
    sample_question["answer_text"],
    max_length=32,
    padding = "max_length",
    truncation = True,
    return_attention_mask = True,
    add_special_tokens = True,
    return_tensors="pt"
)

In [ ]:
tokenizer.decode(answer_encoding["input_ids"].squeeze())

In [ ]:
labels = answer_encoding["input_ids"]
labels

In [ ]:
labels[labels ==0] = -100

In [ ]:
labels

In [ ]:
class ARCQADataset(Dataset):

  def __init__(
    self, 
    data: pd.DataFrame,
    tokenizer: T5Tokenizer,
    source_max_token_len: int = 396,
    target_max_token_len: int = 32  
  ):

    self.tokenizer = tokenizer
    self.data = data
    self.source_max_token_len = source_max_token_len
    self.target_max_token_len = target_max_token_len

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index:int):
    data_row = self.data.iloc[index]

    source_encoding = tokenizer(
        data_row["question"],
        data_row["context"],
        max_length=self.source_max_token_len,
        padding="max_length",
        truncation="only_second",
        return_attention_mask = True,
        add_special_tokens = True,
        return_tensors = "pt"
    )

    target_encoding = tokenizer(
        sample_question["question"],
        sample_question["context"],
        max_length=self.source_max_token_len,
        padding="max_length",
        truncation="only_second",
        return_attention_mask = True,
        add_special_tokens = True,
        return_tensors = "pt"
    )

    labels = target_encoding["input_ids"]
    labels[labels==0] = -100

    return dict(
        question = data_row["question"],
        context = data_row["context"],
        answer_text = data_row["answer_text"],
        input_ids = source_encoding["input_ids"].flatten(),
        attention_mask = source_encoding["attention_mask"].flatten(),
        labels = labels.flatten()
    )

In [ ]:
sample_dataset = ARCQADataset(df, tokenizer)

In [ ]:
for data in sample_dataset:
  print(data["question"])
  print(data["answer_text"])
  print(data["input_ids"][:10])
  print(data["labels"][:10])
  break

#### 6. Train and Validate Datasets

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.5)

In [ ]:
train_df.shape, val_df.shape

In [ ]:
class ARCQADataModule(pl.LightningDataModule):

  def __init__(
      self,
      train_df: pd.DataFrame,
      test_df: pd.DataFrame,
      tokenizer: T5Tokenizer,
      batch_size: int = 8,
      source_max_token_len = 396,
      target_max_token_len = 32
    ):
      super().__init__()
      self.batch_size = batch_size
      self.train_df = train_df
      self.test_df = test_df
      self.tokenizer = tokenizer
      self.source_max_token_len = source_max_token_len
      self.target_max_token_len = target_max_token_len

  def setup(self, stage=None):
    self.train_dataset = ARCQADataset(
        self.train_df,
        self.tokenizer,
        self.source_max_token_len,
        self.target_max_token_len
    )

    self.test_dataset = ARCQADataset(
        self.test_df,
        self.tokenizer,
        self.source_max_token_len,
        self.target_max_token_len
    )

  def train_dataloader(self):
    return DataLoader(
        self.train_dataset,
        batch_size = self.batch_size,
        shuffle = True,
        num_workers = 4
    )

  def val_dataloader(self):
    return DataLoader(
        self.train_dataset,
        batch_size = 1,
        shuffle = True,
        num_workers = 4  
    )

  def test_dataloader(self):
    return DataLoader(
        self.train_dataset,
        batch_size = 1,
        shuffle = True,
        num_workers = 4  
    )

In [ ]:
BATCH_SIZE = 1
N_EPOCHS = 6

data_module = ARCQADataModule(train_df, val_df, tokenizer, batch_size = BATCH_SIZE)
data_module

In [ ]:
data_module.setup()

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)

In [ ]:
model.config

In [ ]:
output = model(
    input_ids = encoding["input_ids"],
    attention_mask = encoding["attention_mask"],
    labels=labels
)

In [ ]:
output.logits.shape

In [ ]:
output.loss

#### 7. Modelling

In [ ]:
class ARCQAModel(pl.LightningModule):

  def __init__(self):
    super().__init__()
    self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)

  def forward(self, input_ids, attention_mask, labels=None):
    output = self.model(
        input_ids = encoding["input_ids"],
        attention_mask = encoding["attention_mask"],
        labels=labels
    )

    return output.loss, output.logits

  def training_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return loss

  def validation_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss 

  def test_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss

  def configure_optimizers(self):
    return AdamW(self.parameters(), lr=0.0001)

In [ ]:
model = ARCQAModel()

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath = "checkpoints",
    filename = "best-checkpoint",
    save_top_k = 1,
    verbose = True,
    monitor = "val_loss",
    mode = "min"
)

In [ ]:
trainer = pl.Trainer(
    callbacks=[checkpoint_callback],
    max_epochs= N_EPOCHS,
    #gpus=1,
    progress_bar_refresh_rate=30
)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir ./lightning_logs

In [ ]:
!rm -rf lightning_logs

In [ ]:
trainer.fit(model,data_module)

In [ ]:
trainer.test()

#### 8. Prediction

In [ ]:
trained_model = ARCQAModel.load_from_checkpoint("checkpoints/best-checkpoint.ckpt")
trained_model.freeze()

In [ ]:
def generate_answer(question):
  source_encoding = tokenizer(
      question["question"],
      question["context"],
      max_length = 396,
      padding="max_length",
      truncation = "only_second",
      return_attention_mask = True,
      add_special_tokens = True,
      return_tensors = "pt"
  )

  generated_ids = trained_model.model.generate(
      input_ids = source_encoding["input_ids"],
      attention_mask = source_encoding["attention_mask"],
      num_beams = 1,
      max_length = 80,
      repetition_penalty = 2.5,
      length_penalty = 1.0,
      early_stopping = True,
      use_cache = True
  )

  preds = [
      tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
      for generated_id in generated_ids
  ]

  return "".join(preds)

In [ ]:
sample_question = val_df.iloc[0]

In [ ]:
sample_question["question"]

In [ ]:
sample_question["answer_text"]

In [ ]:
generate_answer(sample_question)

In [ ]:
sample_question = val_df.iloc[1]

In [ ]:
sample_question["question"]

In [ ]:
sample_question["answer_text"]

In [ ]:
generate_answer(sample_question)

In [ ]:
sample_question = train_df.iloc[12]

In [ ]:
sample_question["question"]

In [ ]:
sample_question["answer_text"]

In [ ]:
sample_question = train_df.iloc[3]

In [ ]:
sample_question["question"]

In [ ]:
sample_question["answer_text"]